# Programming Assignment 1 Graph Algorithms
<br>
Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.
<br>
Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes). (Note also that your answer should not have any spaces in it.)

## Preparation

In [79]:
#various common imports
import numpy as np
import sys
import pandas as pd
import random as rnd
import copy
#interesting to create more easily dictionaries, from https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary
from collections import defaultdict
#import resource
import threading

In [70]:
#Careful recursion and stack preparation. Apparently absolutely necessary
sys.setrecursionlimit(4000)
#hardlimit = resource.getrlimit(resource.RLIMIT_STACK)[1]

### File import
<br>
The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11^{th}11 
th row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

In [2]:
input_file = 'SCC.txt'
with open(input_file, 'r') as data:
    line = data.read().strip().split("\n")
#produces correctly elements with 2 values

In [6]:
len(line)

5105043

In [14]:
line[1].strip()

'1 2'

In [13]:
line[1].split()

['1', '2']

Maybe turn into a **tuple** ?
<br>
Suggested to represent as adjacency list
<br>

In [25]:
prbNet=[[int(s) for s in lin.split()] for lin in line]

In [32]:
#Adjacency list
prbNet2=np.array(prbNet)
uniqueID=np.unique(prbNet2)
print(uniqueID[:30])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30]


In [85]:
#All unique values in the head of vertices
uniqueHead=np.unique(prbNet2[:,0])

In [86]:
len(uniqueHead)

739454

In [77]:
uniqueID[0]

1

In [87]:
#All unique values in the tail of vertices
uniqueTail=np.unique(prbNet2[:,1])

In [88]:
len(uniqueTail)

714547

In [76]:
list(range(4))

[0, 1, 2, 3]

In [78]:
print(uniqueID==list(range(1,875715)))

[ True  True  True ...,  True  True  True]


In [89]:
diffinHead=np.setdiff1d(uniqueHead,uniqueTail)

In [90]:
len(diffinHead)

161167

In [92]:
diffinHead[:10]

array([ 253,  435,  637, 1181, 1218, 1254, 1522, 1656, 1904, 2136])

In [91]:
diffinTail=np.setdiff1d(uniqueTail,uniqueHead)

In [93]:
len(diffinTail)

136260

In [94]:
diffinTail[:10]

array([  4,  21,  37,  39, 108, 150, 174, 223, 230, 232])

There is a clear difference in the size of the two, so <br>
- Value that are unique in the heads,but are not in tails are nodes from which the graph is only outgoing
- On the reverse, values that are unique in tails and that are not in heads are nodes from which the graph is only ingoing
<br>
The dictionary generating routine will need to add for the normal graph those that are unique in the tails because they don't ever have their value on the first position of the vertices, and to the reverse graphs those that are unique in the heads,because they don't ever have their value on the second position.
<br>
These should be added or not? Hard to say, as we can still keep the list of uniqueIDs as a list of nodes.

In [41]:
#Remember how the problem is structured
prbNet[:10]

[[1, 1],
 [1, 2],
 [1, 5],
 [1, 6],
 [1, 7],
 [1, 3],
 [1, 8],
 [1, 4],
 [2, 47646],
 [2, 47647]]

In [44]:
prbNet2[prbNet2[:,0] == 2,1].tolist()

[47646, 47647, 13019, 47648, 47649, 47650, 7700, 47651, 47652]

In [46]:
list(filter(lambda x : x[0]==2,prbNet))

[[2, 47646],
 [2, 47647],
 [2, 13019],
 [2, 47648],
 [2, 47649],
 [2, 47650],
 [2, 7700],
 [2, 47651],
 [2, 47652]]

In [48]:
#Quite an interesting form of high performance container https://docs.python.org/2/library/collections.html#collections.defaultdict

In [57]:
test=defaultdict(list)
print(test)
test['D'].append(5)
print(test)
test['D'].append(6)
print(test)
test['C'].append(8)
print(test)
print(test['D'])
print(type(test['D']))

defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>, {'D': [5]})
defaultdict(<class 'list'>, {'D': [5, 6]})
defaultdict(<class 'list'>, {'D': [5, 6], 'C': [8]})
[5, 6]
<class 'list'>


In [84]:
for i in prbNet:
    if len(i)<2:
        print('node',i, "is strange")

In [59]:
prbDict=defaultdict(list)
for i in prbNet:
    prbDict[str(i[0])].append(i[1])

In [60]:
prbDict['2']

[47646, 47647, 13019, 47648, 47649, 47650, 7700, 47651, 47652]

In [95]:
#Test with one that should have no ingoing
prbDict[str(diffinHead[0])]

[254, 255, 256]

In [96]:
list(filter(lambda x : x[0]==diffinHead[0],prbNet))

[[253, 254], [253, 255], [253, 256]]

In [63]:
#Let's check out if it also works for reversal
#Want to see ALL nodes that go to 9, instead of where 9 goes to
list(filter(lambda x : x[1]==9,prbNet))

[[5, 9],
 [11, 9],
 [26, 9],
 [32, 9],
 [71106, 9],
 [71107, 9],
 [71110, 9],
 [71112, 9],
 [104769, 9],
 [104773, 9],
 [104807, 9],
 [115331, 9],
 [280789, 9],
 [292544, 9],
 [297779, 9],
 [467260, 9],
 [547599, 9],
 [675019, 9],
 [732157, 9],
 [832536, 9],
 [851820, 9],
 [874197, 9]]

In [64]:
#where 9 goes to
list(filter(lambda x : x[0]==9,prbNet))

[[9, 71107],
 [9, 71108],
 [9, 71109],
 [9, 21],
 [9, 71110],
 [9, 71111],
 [9, 39350],
 [9, 71112],
 [9, 71113]]

In [66]:
graphRev=defaultdict(list)
for i in prbNet:
    graphRev[str(i[1])].append(i[0])

In [67]:
graphRev['9']

[5,
 11,
 26,
 32,
 71106,
 71107,
 71110,
 71112,
 104769,
 104773,
 104807,
 115331,
 280789,
 292544,
 297779,
 467260,
 547599,
 675019,
 732157,
 832536,
 851820,
 874197]

In [80]:
len(graphRev)

714547

### Test Cases
<br>
Test cases imported from the forums


In [18]:
test_Dict={
    'A':[[1, 4],[2, 8],[3, 6],[4, 7],[5, 2],[6, 9],[7, 1],[8, 5],[8, 6],[9, 7],[9, 3]],
    'B':[[1, 2],[2, 6],[2, 3],[2, 4],[3, 1],[3, 4],[4, 5],[5, 4],[6, 5],[6, 7],[7, 6],[7, 8],[8, 5],[8, 7]],
    'C':[[1, 2],[2, 3],[3, 1],[3, 4],[5, 4],[6, 4],[8, 6],[6, 7],[7, 8]],
    'D':[[1, 2],[2, 3],[3, 1],[3, 4],[5, 4],[6, 4],[8, 6],[6, 7],[7, 8],[4, 3],[4, 6]],
    'E':[[1, 2],[2, 3],[2, 4],[2, 5],[3, 6],[4, 5],[4, 7],[5, 2],[5, 6],[5, 7],[6, 3],[6, 8],[7, 8],[7, 10],[8, 7],[9, 7],[10, 9],[10, 11]
        ,[11, 12],[12, 10]]
}

ans_Dict={
    'A':[3,3,3,0,0],
    'B':[3,3,2,0,0],
    'C':[3,3,1,1,0],
    'D':[7,1,0,0,0],
    'E':[6,3,2,1,0]
}

In [36]:
#Turn into adjacency list
#First get all unique values
testIDs=[]
for i, k in test_Dict.items():
    testIDs.append(np.unique(np.array(k)).tolist())
print(testIDs)

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]


In [ ]:
#Turn all lists into dictionaries

## Actual Code

Look at
https://github.com/ladamalina/coursera-algo/blob/master/PQ4.%20SCCs/kosaraju.py
<br>


In [ ]:
#Define

## Run cases
<br>
Let's try it first
### Test Cases

In [ ]:
#Test cases

### Problem Case
<br>
Some suggestions from the forum

In [ ]:
sys.setrecursionlimit(800000)
threading.stack_size(67108864)

def main():
  '''
  YOUR CODE HERE
  '''
thread = threading.Thread(target=main)
thread.start()